In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
train_datagen = ImageDataGenerator(rescale=1.0/255, 
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [3]:
# Áp dụng data augmentation cho tập huấn luyện
train_generator = train_datagen.flow_from_directory(directory="D:\data\Ai trong kinh doannh\Dự án AI\Tomato_train",
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical')

# Tạo bộ tăng cường dữ liệu cho tập validation (không thay đổi dữ liệu)
validation_datagen = ImageDataGenerator(rescale=1.0/255)

# Áp dụng data augmentation cho tập validation
validation_generator = validation_datagen.flow_from_directory(directory="D:\data\Ai trong kinh doannh\Dự án AI\Tomato_test",
                                                              target_size=(150, 150),
                                                              batch_size=32,
                                                              class_mode='categorical')

Found 707 images belonging to 3 classes.
Found 0 images belonging to 0 classes.


In [4]:
# Khởi tạo mô hình CNN
model = Sequential()

# Các lớp Convolutional
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

# Flatten
model.add(Flatten())

# Fully-connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(3, activation='softmax'))

In [5]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [6]:
# Compile mô hình
model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Sử dụng EarlyStopping để dừng quá trình huấn luyện sớm nếu không có cải tiến đáng kể
early_stopping = EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True)

In [7]:
# Huấn luyện mô hình
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.n // train_generator.batch_size,
                    epochs=20,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.n // validation_generator.batch_size)

Epoch 1/20
22/22 [==============================] - 24s 990ms/step - loss: 0.9803 - accuracy: 0.5185
Epoch 2/20
22/22 [==============================] - 11s 503ms/step - loss: 0.7104 - accuracy: 0.6281
Epoch 3/20
22/22 [==============================] - 11s 497ms/step - loss: 0.6591 - accuracy: 0.6519
Epoch 4/20
22/22 [==============================] - 11s 483ms/step - loss: 0.5773 - accuracy: 0.6622
Epoch 5/20
22/22 [==============================] - 11s 493ms/step - loss: 0.5789 - accuracy: 0.6741
Epoch 6/20
22/22 [==============================] - 11s 492ms/step - loss: 0.5282 - accuracy: 0.6859
Epoch 7/20
22/22 [==============================] - 11s 484ms/step - loss: 0.5761 - accuracy: 0.6696
Epoch 8/20
22/22 [==============================] - 11s 479ms/step - loss: 0.5064 - accuracy: 0.7111
Epoch 9/20
22/22 [==============================] - 11s 497ms/step - loss: 0.5034 - accuracy: 0.7170
Epoch 10/20
22/22 [==============================] - 11s 515ms/step - loss: 0.5189 - accura

In [8]:
# Đánh giá độ chính xác của mô hình
score = model.evaluate(train_generator,verbose=0)
print('Sai số kiểm tra là: ',score[0])
print('Độ chính xác kiểm tra là: ',score[1])

Sai số kiểm tra là:  0.3457064926624298
Độ chính xác kiểm tra là:  0.8472418785095215


In [13]:
model_CNN = load_model('Tomato_ripeness01.h5')
from tensorflow.keras.utils import load_img, img_to_array
import matplotlib.pyplot as plt
import numpy as np
filename = "/kaggle/input/tomato-test/Tomato_test/IMG_0993.jpg"

predict = ['Chua chin','Gan chin','Da chin']
predict = np.array(predict)
img = load_img(filename,target_size=(150,150))
plt.imshow(img)

img = img_to_array(img)
img = img.reshape(1,150,150,3)
img = img.astype('float32')
img = img/255


result = np.argmax(model_CNN.predict(img),axis=-1)
predict[result]

NameError: name 'load_model' is not defined